# Trabalho final - Branch And Bound - Pesquisa Operacional
## Alunos: Guilherme Iram, Tales Nobre
## Professor: Teobaldo Bulhões 

In [1]:
from mip import *

In [2]:
def solve(model):
    status = model.optimize()
    
    if str(status) == "OptimizationStatus.INFEASIBLE":
        print("INVIÁVEL")
        return
    
    print("Status = ", status)
    print(f"Solution value  = {model.objective_value:.2f}\n")
  
    print("Solution:")
    for v in model.vars:
        print(f"{v.name} = {v.x:.2f}")

In [3]:
def print_modelo(model):
    model.write("model.lp")
    with open("model.lp") as f: 
        print(f.read())

In [4]:
def is_int(n):
    return n % 1 == 0

In [5]:
def objectivo_value_cost(node):
    node.model.optimize()
    return node.model.objective_value

In [6]:
def read_instance(path):
    file = open(path, 'r')
    n_var = 0
    n_restricoes = 0
    coeficientes_fo = []
    restricoes = []

    for i, line in enumerate(file.readlines()):

        if i == 0:
            n_var, n_restricoes = line.split(" ")
            n_var, n_restricoes = int(n_var), int(n_restricoes)

        elif i == 1:
            for coef in line.split(" "):
                coeficientes_fo.append(float(coef))

        else:
            coeficientes = []

            for n in line.split(" "):
                coeficientes.append(float(n))

            restricoes.append(coeficientes)

    model = Model(sense=MAXIMIZE, solver_name=CBC)

    x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i + 1}", lb=0.0, ub=1.0) for i in range(n_var)]

    model.objective = xsum([coeficientes_fo[i] * x[i] for i in range(n_var)])
    
    for restricao in restricoes:
        LHS = xsum([restricao[i] * x[i] for i in range(n_var)])
        model += LHS <= restricao[-1]
    
    return model, x


In [7]:
class Node():
    def __init__(self, model, x, dual_limit = 0, primal_limit = 0, parent=None, left_node=None, rigth_node=None):
        self.model = model
        self.x = x
        self.dual_limit = dual_limit
        self.primal_limit = primal_limit
        self.parent = parent
        self.left = left_node
        self.right = rigth_node

In [8]:
def build_tree(node, x, primal_limit, dual_limit, best_nodes_list):
    
    status = node.model.optimize()
    
    # poda por inviabilidade
    if str(status) == "OptimizationStatus.INFEASIBLE":
        node.left = None
        node.right = None
        # print("Podei por INVIABILIDADE")
        return
    
    fo_value = node.model.objective_value
    
    # atualizando o limite dual
    if fo_value > dual_limit:
        dual_limit = fo_value
        
    # poda por limitante
    if primal_limit > fo_value:
        node.left = None
        node.right = None
        # print("Podei por LIMITANTE")
        return
    
   
    all_int = True
    for v in node.model.vars:
        if not is_int(v.x):
            all_int = False
            
    # poda por integralidade
    if all_int:
        # atualizacao do limite primal
        if primal_limit < fo_value:
            primal_limit = fo_value
            best_nodes_list.append(node)
            node.left = None
            node.right = None
            # print("Podei por INTEGRALIDADE")
        return
        
    for i, v in enumerate(node.model.vars):
        
        if not is_int(v.x):
            aux1 = node.model.copy()
            aux2 = node.model.copy()
            
            aux1 += x[i] == 0
            aux2 += x[i] == 1
            
            aux1.optimize()
            aux2.optimize()
            
            node_1 = Node(aux1, x, dual_limit, primal_limit, node, None, None)
            node_2 = Node(aux2, x, dual_limit, primal_limit, node, None, None)
            
            node.left = node_1
            node.right = node_2
            
            build_tree(node_1, node_1.x, primal_limit, dual_limit, best_nodes_list)
            build_tree(node_2, node_2.x, primal_limit, dual_limit, best_nodes_list)
            
    return best_nodes_list

In [ ]:
modelo, x = read_instance("instancias//instancia_5.txt")

aux1 = modelo.copy()    
aux1.optimize()
node = Node(model=aux1, x=x, dual_limit = aux1.objective_value, primal_limit = 0,parent=None, left_node=None, rigth_node=None)

list_nodes = build_tree(node, x, node.primal_limit, node.dual_limit, [])

In [ ]:
best_node = list_nodes[0]

for node in list_nodes:
    if objectivo_value_cost(node) > objectivo_value_cost(best_node):
        best_node = node

In [ ]:
print_modelo(best_node.model)

In [ ]:
solve(best_node.model)